In [1]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd

def checker(wrong_options,correct_options):

    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
            
           
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

df_Original_List=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Supplier Score Card\New folder\Commercial.xlsx')

df_To_beMatched=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Supplier Score Card\New folder\Audits.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df_Original_List['Supplier_Name_3']=df_Original_List['Parent_Supplier_name'].str.upper()
df_To_beMatched['Supplier_Name_3']=df_To_beMatched['Supplier Name'].str.upper()


df_Original_List['Supplier_Name_3']=df_Original_List['Supplier_Name_3'].str.replace('LTD','').str.replace('LIMITED','')
df_To_beMatched['Supplier_Name_3']=df_To_beMatched['Supplier_Name_3'].str.replace('LTD','').str.replace('LIMITED','')


In [4]:
audits_matched=pd.read_excel(r'C:\Users\IN22912959\Audits_Matched.xlsx')
audits_matched.head(2)

,Unnamed: 0,Supplier Code,Supplier Name_x,Site Code_x,Site Name_x,Audit/Visit Name,Latest Audit Date,Latest Audit Score,Score,Due Date,...,Category_Director,Commercial_Director,Parent_Supplier,Parent_Supplier_name,Supplier_Number,Supplier_Name,Supplier Name_y,Site Code_y,Site Name_y,Fuzzy_Ratio
0,2,67880,Onubafruit,ON5,Costa de Huelva,Visit: Pre-approval,2018-04-06,Green,Amber,2017-02-16,...,PRODUC E.,FRES H.,10068,ONUBAFRUIT SL,67880,"ONUBAFRUIT,S.COOP.AND.(FF",Onubafruit,ON5,Costa de Huelva,Easy Join
1,3,67880,Onubafruit,ON5,Costa de Huelva,Visit: Pre-approval,2018-04-06,Green,Amber,2017-02-16,...,PRODUC E.,FRES H.,10068,ONUBAFRUIT SL,67880,"ONUBAFRUIT,S.COOP.AND.(FF",Onubafruit,ON6,Cartaya 1,Easy Join


In [5]:
not_matched=df_To_beMatched.merge(audits_matched[['Supplier Code','Junior_Buyer','Fuzzy_Ratio']].drop_duplicates(),how='left',left_on=['Supplier Code','Junior_Buyer'],right_on=['Supplier Code','Junior_Buyer'])
not_matched.head(2)

,Supplier Code,Supplier Name,Site Code,Site Name,Audit/Visit Name,Latest Audit Date,Latest Audit Score,Score,Due Date,From Date,...,list=Business Units,Path,Lead Technical Manager,Country,Junior_Buyer,Buying_Controller,Category_Director,Commercial_Director,Supplier_Name_3,Fuzzy_Ratio
0,A6900,HB Gold Standard Supplier,5010692,HB Gold Standard Site 1,Audit: Training Module Maintenance - only to b...,2017-11-08,NaN,Blue,2017-01-09,2017-01-08,...,United Kingdom,TRAINING / Training,UK Helen Boley,United Kingdom,AUSTRALIA O.,WINE S.,BW S.,PACKAGE D.,HB GOLD STANDARD SUPPLIER,NaN
1,A6900,HB Gold Standard Supplier,5010692,HB Gold Standard Site 1,Audit: Training Module Approval - only to be u...,2017-11-08,NaN,NaN,2017-04-27,2017-04-27,...,United Kingdom,TRAINING / Training,ZZ Training User,United Kingdom,AUSTRALIA O.,WINE S.,BW S.,PACKAGE D.,HB GOLD STANDARD SUPPLIER,NaN


In [9]:
not_matched=not_matched[not_matched['Fuzzy_Ratio'].isnull()]
not_matched.head(2),len(not_matched)

(  Supplier Code              Supplier Name Site Code                Site Name  \
 0         A6900  HB Gold Standard Supplier   5010692  HB Gold Standard Site 1   
 1         A6900  HB Gold Standard Supplier   5010692  HB Gold Standard Site 1   
 
                                     Audit/Visit Name Latest Audit Date  \
 0  Audit: Training Module Maintenance - only to b...        2017-11-08   
 1  Audit: Training Module Approval - only to be u...        2017-11-08   
 
   Latest Audit Score Score   Due Date  From Date  ... list=Business Units  \
 0                NaN  Blue 2017-01-09 2017-01-08  ...      United Kingdom   
 1                NaN   NaN 2017-04-27 2017-04-27  ...      United Kingdom   
 
                   Path Lead Technical Manager         Country  \
 0  TRAINING / Training         UK Helen Boley  United Kingdom   
 1  TRAINING / Training       ZZ Training User  United Kingdom   
 
        Junior_Buyer Buying_Controller Category_Director Commercial_Director  \
 0  AUSTR

In [13]:
jb_comm=not_matched['Junior_Buyer'].unique()
jb_comm=pd.DataFrame(jb_comm,columns={'Junior_Buyer'})
# jb_comm=jb_comm[jb_comm['Junior_Buyer']=='STONEFRUI     T.']
jb_comm

,Junior_Buyer
0,AUSTRALIA O.
1,FROZEN PIE S.
2,FRZEN MEAT FREE.
3,FROZENDESSERT S.
4,ISB BREA D.
...,...
151,TURKE Y.
152,FRANCE E.
153,CATLITTE R.
154,EASY ENTRTAIN G.


In [14]:
cols=['TQC_Supplier_Name','Supplier_Code','Comm_Supplier_Name','junio_buyer','correct_ratio']
# commercial_for_loop=commercial_only_OBL.copy()
df2=pd.DataFrame()

output_list=[]


for i in jb_comm['Junior_Buyer']:
    print(i)
    filter_data_tqc=df_To_beMatched[df_To_beMatched['Junior_Buyer']==i]
    filter_data_comm=df_Original_List[df_Original_List['Junior_Buyer']==i]

    if not filter_data_comm.empty:

        str2Match = filter_data_tqc['Supplier_Name_3'].fillna('######').tolist()
        strSupplier_Code =filter_data_tqc['Supplier Code'].tolist()
        strOptions =filter_data_comm['Supplier_Name_3'].fillna('######').tolist()
#         strTesco_Brand_Ind =filter_data_tqc['Tesco_Brand_Ind'].fillna('######').tolist()
        
#         name_match,ratio_match=checker(str2Match,strOptions)
        
        for j in range(0,len(str2Match)):
            x=process.extractOne(str2Match[j],strOptions,scorer=fuzz.token_set_ratio)
            while x[1]==100:
                output_list=[str2Match[j],strSupplier_Code[j],x[0],i,x[1]]
                zipped = zip(cols, output_list)
                a_dictionary = dict(zipped)
                df1=pd.DataFrame(a_dictionary,columns=cols,index=[j])
                df2=df2.append(df1)
                #drop row which has the matched supplier name and junor buyer
                strOptions.remove(x[0])
                x=process.extractOne(str2Match[j],strOptions,scorer=fuzz.token_set_ratio)
            strOptions =filter_data_comm['Supplier_Name_3'].fillna('######').tolist()
#                 output_list=[str2Match[j],strSupplier_Code[j],x[0],i,x[1],strTesco_Brand_Ind[j]]
                
#                 filter_data_comm.drop(filter_data_comm[(filter_data_comm['Supplier_Name_3'] ==x[0])&(filter_data_comm['Junior_Buyer'] ==i)].index, inplace = True) 
                
            output_list=[str2Match[j],strSupplier_Code[j],x[0],i,x[1]]
            zipped = zip(cols, output_list)
            a_dictionary = dict(zipped)
            df1=pd.DataFrame(a_dictionary,columns=cols,index=[j])
            df2=df2.append(df1)

        
        
df2.head(2)


AUSTRALIA     O.
FROZEN PIE    S.
FRZEN MEAT FREE.
FROZENDESSERT S.
ISB BREA      D.
MORNING GOOD  S.
FBAKERYORDERABL.
ISB SWEE      T.
PACKETSNACK   S.
RIC           E.
CANNEDSOU     P.
HOMEBAK       E.
PACKETMIXE    S.
PORIDGE&ONTHEGO.
CANNEDVE      G.
TRAD MIXE     S.
ORALCAR       E.
COOKEDCHICKE  N.
FRES          H.
BREA          D.
COUNTERPIZZ   A.
REGULAR CAKE  S.
SEASONALCAKE  S.
CHILLED PIE   S.
BURGER&MEATBALL.
CHILLED SNACK S.
MFP PIE       S.
FRONT OF STOR E.
TABLESAUCE    S.
PAN ASIAN     G.
CONDSALADDRES S.
SOUP SAUCE PA S.
BERRIE        S.
ORGANICVE     G.
AMBIENT SALA  D.
CNTR POULTR   Y.
DELI COUNTE   R.
AMBIENTVE     G.
CHILLEDVE     G.
SPECIALITYVE  G.
CHILLED SALA  D.
COUNTERSMEA   T.
CHICKE        N.
READY TO COO  K.
POR           K.
CORE BEE      F.
COREMEAL      S.
PREPVE        G.
VEG MEALS & ING.
EXPRESSFLOWER S.
CORE&SEASONA  L.
EVENTFLOWER   S.
PREPSALA      D.
DRIEDFRUITNUT S.
MULTICRISP    S.
SEASONBIGBAGC S.
SWEETBISCUIT  S.
CHILLED PARTY F.
FROZEN PARTY  

,TQC_Supplier_Name,Supplier_Code,Comm_Supplier_Name,junio_buyer,correct_ratio
0,HB GOLD STANDARD SUPPLIER,A6900,PALMER AND HARVEY DIST,AUSTRALIA O.,43
1,HB GOLD STANDARD SUPPLIER,A6900,PALMER AND HARVEY DIST,AUSTRALIA O.,43


In [15]:
df3=df2[df2['correct_ratio']>=80].drop_duplicates()
df3.head(2)

,TQC_Supplier_Name,Supplier_Code,Comm_Supplier_Name,junio_buyer,correct_ratio
15,ORIGIN WINE .,62500,ORIGIN WINE,AUSTRALIA O.,100
18,EQUIPE SRL,61416,EQUIPE SRL,AUSTRALIA O.,100


In [16]:
comm=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Supplier Score Card\New folder\Commercial.xlsx')
comm.head(2)
comm['Supplier_Name_3']=comm['Supplier_Name'].str.upper().str.replace('LTD','').str.replace('LIMITED','')
comm.head(2)

,Parent_Supplier,Parent_Supplier_name,Supplier_Number,Supplier_Name,Commercial_Director,Commercial_Area,Category_Director,Category_Area,Buying_Controller,Buyer_Area,Buyer,Junior_Buyer,Junior_Area,Product_Area,Status,COGS_TY,OBL_Count,BL_Count,Supplier_Name_3
0,4723,OTL,65450,OTL,PACKAGE D.,Packaged,HB&WELLNES S.,"Health, Beauty & Wellness",TOILETRIE S.,Bathing & Washing,BATHING&WASHING.,WASHIN G.,Washing,Toiletries,Yes,19638.919,0,216,OTL
1,6404,GOLDEN ACRE DAIRY FOODS LTD,63057,GOLDEN ACRE FOODS LTD,PACKAGE D.,Packaged,GROCER Y.,Grocery,INGREDS&GLOBA L.,"European, USA, Caribbean & Kos",EUR USA CAR K O.,KOSHE R.,Kosher,Ingredients & Global Grocery,Yes,76057.291,0,3617,GOLDEN ACRE FOODS


In [17]:
match=comm.merge(df3,how='inner',right_on=['Comm_Supplier_Name','junio_buyer'],left_on=['Supplier_Name_3','Junior_Buyer'])
match.head(2)

,Parent_Supplier,Parent_Supplier_name,Supplier_Number,Supplier_Name,Commercial_Director,Commercial_Area,Category_Director,Category_Area,Buying_Controller,Buyer_Area,...,Status,COGS_TY,OBL_Count,BL_Count,Supplier_Name_3,TQC_Supplier_Name,Supplier_Code,Comm_Supplier_Name,junio_buyer,correct_ratio
0,194,WALKERS CHOCOLATES,47850,WALKERS CHOCOLATES,PACKAGE D.,Packaged,IMPULS E.,Impulse,CONFECTIONER Y.,Sweets & Gums,...,Yes,5.125530e+06,4262,0,WALKERS CHOCOLATES,WALKERS CHOCOLATES,47850,WALKERS CHOCOLATES,SWEETS&GUM S.,100
1,6821,GROVE TURKEYS LTD,38422,GROVE TURKEYS LTD,FRES H.,Fresh,MF P.,MFP,POULTRYANDEGG S.,Turkeyduckgame,...,Yes,1.805125e+07,3219,189,GROVE TURKEYS,GROVE TURKEYS,62623,GROVE TURKEYS,TURKE Y.,100


In [18]:
aud_sites=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Supplier Score Card\New folder\Audits.xlsx')
aud_sites.head(2)

,Supplier Code,Supplier Name,Site Code,Site Name,Audit/Visit Name,Latest Audit Date,Latest Audit Score,Score,Due Date,From Date,...,Audits and Visits Status,Site Status,list=Business Units,Path,Lead Technical Manager,Country,Junior_Buyer,Buying_Controller,Category_Director,Commercial_Director
0,A6900,HB Gold Standard Supplier,5010692,HB Gold Standard Site 1,Audit: Training Module Maintenance - only to b...,2017-11-08,NaN,Blue,2017-01-09,2017-01-08,...,Awaiting Sign-Off,Active,United Kingdom,TRAINING / Training,UK Helen Boley,United Kingdom,AUSTRALIA O.,WINE S.,BW S.,PACKAGE D.
1,A6900,HB Gold Standard Supplier,5010692,HB Gold Standard Site 1,Audit: Training Module Approval - only to be u...,2017-11-08,NaN,NaN,2017-04-27,2017-04-27,...,In Progress,Active,United Kingdom,TRAINING / Training,ZZ Training User,United Kingdom,AUSTRALIA O.,WINE S.,BW S.,PACKAGE D.


In [19]:
aud_sites=aud_sites[['Supplier Code','Supplier Name','Site Code','Site Name','Junior_Buyer']].drop_duplicates()
aud_sites.head(2)

,Supplier Code,Supplier Name,Site Code,Site Name,Junior_Buyer
0,A6900,HB Gold Standard Supplier,5010692,HB Gold Standard Site 1,AUSTRALIA O.
2,67880,Onubafruit,ON5,Costa de Huelva,BERRIE S.


In [20]:
match_2=match.merge(aud_sites,how='inner',left_on=['Supplier_Code','Junior_Buyer'],right_on=['Supplier Code','Junior_Buyer'])

In [21]:
match_2.to_excel('Parent Name to name matching.xlsx')